In [1]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras import backend as K

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
# Скачать данные можно по этой ссылке: https://drive.google.com/drive/folders/1ewl6A5neZ5tFduwud186b011qDQ_rzgE?usp=sharing

In [2]:
test = pd.read_csv('sample_submission.csv')
test.head()

,image_path,emotion
0,0.jpg,neutral
1,1.jpg,neutral
2,2.jpg,neutral
3,3.jpg,neutral
4,4.jpg,neutral


In [3]:
class emotion_model():
    '''
        Для загрузки предобученных весов в метод __init__ 
        необходимо передать аргумент с путем до файла с моделью в формате .h5
        path_to_model = '/path/to/weights.h5'

        Входящее изображение должно быть размера 224х224 
        в формате rgb
        input_shape=(224,224,1)

        Выходящий тензор будет иметь вероятности для 9 классов
        output_shape=(9)
        
        метод get_emotion_from_the_image_path() получает путь до изображение
        и возвращает предсказанную эмоцию
        
        метод get_emotion_from_the_directory_path() получает датафрейм с названиями фото и
        путь до директории с изображениями и возвращает предсказанную эмоцию
        
        метод preprocess_input_facenet() получает изображение и возвращает 
        преобразованное изображение, для более лучшего предсказания эмоции
    '''
    def __init__(self, path_to_model):
        super().__init__()
        self.model = load_model(path_to_model)

        self.emotion_dict = {0:'anger',
                        1:'contempt',
                        2:'disgust',
                        3:'fear',
                        4:'happy',
                        5:'neutral',
                        6:'sad',
                        7:'surprise',
                        8:'uncertain'}
        
    
    def get_emotion_from_the_image_path(self, img_path):
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = self.preprocess_input_facenet(img)
        
        return self.emotion_dict[np.argmax(self.model.predict(img))]
    
    def get_emotion_from_the_directory_path(self, directory_path,dataset):
        BATCH_SIZE = 128
        IMAGE_SIZE = 224
        
        image_gen = ImageDataGenerator(preprocessing_function=self.preprocess_input_facenet)  

        data = image_gen.flow_from_dataframe(
            dataset,
            directory=directory_path,
            x_col='image_path',
            y_col='emotion',
            target_size=(IMAGE_SIZE, IMAGE_SIZE),
            class_mode='categorical',
            batch_size=BATCH_SIZE,
            shuffle=False)
        
        emotion = self.model.predict(data)
        list_of_emotions = [self.emotion_dict[np.argmax(i)] for i in emotion]
        
        return list_of_emotions
        
    def preprocess_input_facenet(self,x):
        x_temp = np.copy(x)
        data_format = K.image_data_format()
        
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
            
        return x_temp    
    
    

In [4]:
model = emotion_model('model.h5')

In [ ]:
test['emotion'] = model.get_emotion_from_the_directory_path('test_kaggle',test)
test.head()